
---
# Processing passive seismic data

This notebook has some practical processing activities of the Course **ProSeisSN**. It deals with data processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib

#### Reset the Jupyter/IPython notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***

In [ ]:
# If needed for mathplotlib
# conda install -c conda-forge ipympl

In [1]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
"""
Activate the ipympl backend by including
%matplotlib ipympl
magic in the notebook or by
%matplotlib widget
which will have the same effect.

Other magic:
%matplotlib inline
%matplotlib notebook
"""
%matplotlib widget

# Import specialized ObsPy packages + sanity.
try:
    import obspy
    print('obspy version==>', obspy.__version__)
except TypeError:
    print('Stopping RUNTIME. ObsPy not found.')
    exit()

from obspy import read
from obspy import UTCDateTime
from obspy import read, Stream
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.header import FDSNException

# Local routines. This is just a memo, does not do anything.
from utils import period2freq, freq2period
_ = freq2period(1.)

ModuleNotFoundError: No module named 'ipympl'

---
## Accessing a local data

This lecture serves as a basic introduction to using Obspy. Useful links are the official [Obspy Tutorial](https://docs.obspy.org/tutorial/index.html), and the [Seismo-Live Juypter Notebooks for Seismology](https://krischer.github.io/seismo_live_build/tree/index.html).

We work with data form an event occured in Acre in 2024, downloaded using [IRIS Wilber](http://www.iris.edu/wilber3).

|EventID | Time | Lat | Long | Depth(km) | Mww | Location |
| :- | :-: | :- | :- | :-: | :-: | :- |
|11793085|2024-01-20T21:31|-7.2879|-71.464|607|6.6|WESTERN BRAZIL|

We will use a data file in miniSEED format: [wb11793085_ir.mseed](https://github.com/jandyr/ProSeisSN_Nbk/blob/main/wb11793085_ir.mseed).

Choose stations with epicentral distances $\left[20^{\circ},\,70^{\circ}\right]$; waves travel in the laterally homogeneous Mantle, with P and S arrivals.

Of the 149 stations operational during the event, select station IU HKT, distant $44^{\circ}$ from the event.

|Station|Seismometer| Lat | Long | Depth(m) | Channels |
| :- | :- | :- | :- | :-: | :-: |
|HKT: Hockley1 Texas|Streckeisen STS-6A VBB |29.96|-95.84|93|BH1, BH2, BHZ|

**Phase Arrivals**

|Phase|$\Delta t$| TIme |
| :- | :-: | :- |
|P|+7m 14s|21:38:19|
|PP|+9m 9s |21:40:14|
|S|+13m 4s|21:44:09|
|SS|+16m 38s|21:47:43|

In [ ]:
# Specify event's start and end times
starttime = UTCDateTime("2024-01-20T21:37:19.019539Z") - 60
endtime = UTCDateTime("2024-01-20T21:58:18.994539Z")
print(starttime, endtime)

# Use wildcards to select all three HH* channels
net = "IU"
sta = "HKT"
loc = "00"
chan = "HH*"

# Specify client. Opt:from obspy.clients.earthworm import Client
# Get waveforms with instrument response into a stream
# List of ObsPy clients: clients = ["IRIS", "NCEDC", "USGS", "GEONET", "RESIF", "INGV", "BGR", "ODC", "SCEDC"]
client = Client("IRIS")
        try:
          st = client.get_waveforms(net, sta, loc, chan, starttime, endtime, attach_response = True)
          print(st)
        except FDSNException:
          print(f"Client " + client + " is not working.")

The first letter is 'y', 'Y', or 'n'.


## Example: Fit a cosine to the data

First define a Python function where the first input argument are the "x" values (an array) and the remaining arguments are the parameters that can be adjusted to find the optimal fit. In this case, a simple cosine function with an adjustable amplitude and frequency is created.

$$ \dot{\theta} = A \cos \omega t $$

In [ ]:
def cos_func(times, amplitude, frequency):
    return amplitude * np.cos(frequency * times)

To check that it works, provide some values:

In [ ]:
times = np.linspace(0, 10)
cos_func(times, 5.0, 5.0)

In [ ]:
fig, ax = plt.subplots()
ax.plot(times, cos_func(times, 5.0, 5.0))

In [ ]:
fig, ax = plt.subplots()
ax.plot(radial_gyro_meas.index, cos_func(radial_gyro_meas.index, 5.0, 5.0))

Now, provide this function to `curve_fit` along with the measure data (x and y) and an initial guess for the amplitude and frequency. A good initial guess is important, as the optimal solution can't always be found from an arbitrary initial guess. The function `curve_fit` returns two items. The first is the optimal values of the two parametes and the second is the covariance matrix that gives an idea of how certain the value of the parameters are. We will just work with the first value for now.

In [ ]:
popt, pcov = curve_fit(cos_func,  # our function
                       radial_gyro_meas.index,  # measured x values
                       radial_gyro_meas.angular_velocity,  # measured y values
                       p0=(3.0, period2freq(0.44)))  # the initial guess for the two parameters

Now we see the optimal values for the amplitude and frequency:

In [ ]:
popt

It is useful to plot the optimal function versus the measured data:

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(radial_gyro_meas.index, radial_gyro_meas.angular_velocity, '.', label='Measured')
ax.plot(radial_gyro_meas.index, cos_func(radial_gyro_meas.index, popt[0], popt[1]), label='Best Fit')
ax.legend()

That looks like a pretty nice fit! The period of this fitted function can now be computed:

In [ ]:
freq2period(popt[1])

## Exercise

This system, just like the book on a cup system has an amplitude that decreases with respect to time. This decrease is often referred to as a *decaying amplitude*. We just fit the mathematical function:

$$ \dot{\theta}(t) = A \cos{\omega t} $$

where $A$ is the amplitude of the angular rate $\dot{\theta}$ and $\omega$ is the frequency of oscillation in radians per second. One way to account for the decay in amplitude oscillation is to introduce a multiplicative factor of $e^{-\lambda t}$ into the equation. This will cause exponential decay at the rate $\lambda$ (units are 1 / s). The mathematical equation will look like:

$$ \dot{\theta}(t) = A e^{-\lambda t} \cos{\omega t} $$

Recall that $e^{-\lambda t}$ looks like:

In [ ]:
fig, ax = plt.subplots()
t = np.linspace(0, 2, num=100)
ax.plot(t, np.exp(-1.5 * t));

Use decaying oscillation mathematical function to create a curve fitting function and find the values of $A$, $\lambda$, and $\omega$ that best fit the data. Calculate the period of oscillation and compare it to the period from the purely sinusoidal fit from above. Is there any difference in the period of oscillation?

In [ ]:
# write solution here

In [ ]:
def decaying_sinusoid(t, a, lam, w):
    return a * np.exp(lam * t) * np.cos(w * t)

popt, pcov = curve_fit(decaying_sinusoid,
                       radial_gyro_meas.index,
                       radial_gyro_meas.angular_velocity,
                       p0=(3.0, -0.0002, period2freq(0.44)))

fig, ax = plt.subplots(1, 1)
ax.plot(radial_gyro_meas.index, radial_gyro_meas, '.')
ax.plot(radial_gyro_meas.index, decaying_sinusoid(radial_gyro_meas.index, popt[0], popt[1], popt[2]));

freq2period(popt[2])